## CSV dosyasındaki verinin yüklenmesini gerçekleştirme

In [1]:

from langchain_community.document_loaders.csv_loader import CSVLoader

filepath = "IMDb_Top_1000_Movies_Dataset.csv"
loader = CSVLoader(filepath, encoding="utf-8")

data = loader.load()

print(f"Toplam {len(data)} film yüklendi.")

c:\Users\Nihal\miniconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Toplam 1000 film yüklendi.


## Veriyi parçalarına ayırma(Chunking işlemi)

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

print(f"Number of documents after splitting: {len(docs)}")
docs[7]

Number of documents after splitting: 1245


Document(metadata={'source': 'IMDb_Top_1000_Movies_Dataset.csv', 'row': 6}, page_content=": 6\nMovie_Name: Schindler's List\nMovie_Rating: 9.0\nMovie_Certificate: A\nMovie_Year: 1993\nMovie_Runtime: 195 min\nMovie_Cast: Liam Neeson, Ralph Fiennes, Ben Kingsley, Caroline Goodall\nMovie_Genre: Biography, Drama, History\nMovie_Poster: https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_UX67_CR0,0,67,98_AL_.jpg\nMovie_Description: In German-occupied Poland during World War II, industrialist Oskar Schindler gradually becomes concerned for his Jewish workforce after witnessing their persecution by the Nazis.\nMovie_Poster_HD: https://m.media-amazon.com/images/M/MV5BNDE4OTMxMTctNmRhYy00NWE2LTg3YzItYTk3M2UwOTU5Njg4XkEyXkFqcGdeQXVyNjU0OTQ0OTY@._V1_.jpg")

## Google Generative AI Embeddings'i kullanarak Embedding Oluşturma İşlemi


In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv


In [4]:
load_dotenv()  # Load environment variables from .env file
import os
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    print("HATA: API anahtarı .env dosyasından okunamadı!")
else:
    print(f"Anahtar başarıyla yüklendi. İlk 4 karakter: {api_key[:4]}")

Anahtar başarıyla yüklendi. İlk 4 karakter: AIza


In [9]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004", task_batch_size=100)
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.014134909, -0.022324152, -0.05460342, -0.0062845494, -0.033924025]

## ChromaDB üzerine kayıt işlemi

In [10]:
from langchain_chroma import Chroma

In [11]:
vector_store = Chroma.from_documents(documents=docs, embedding=embeddings)

In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10 })

In [13]:
retrieved_docs = retriever.invoke("Movies starring Meryl Streep")     

len(retrieved_docs)

print(retrieved_docs[1].page_content)


: 513
Movie_Name: Fantastic Mr. Fox
Movie_Rating: 7.9
Movie_Certificate: U
Movie_Year: 2009
Movie_Runtime: 87 min
Movie_Cast: George Clooney, Meryl Streep, Bill Murray, Jason Schwartzman
Movie_Genre: Animation, Adventure, Comedy
Movie_Poster: https://m.media-amazon.com/images/M/MV5BOGUwYTU4NGEtNDM4MS00NDRjLTkwNmQtOTkwMWMyMjhmMjdlXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg
Movie_Description: An urbane fox cannot resist returning to his farm raiding ways and then must help his community survive the farmers' retaliation.
Movie_Poster_HD: https://m.media-amazon.com/images/M/MV5BOGUwYTU4NGEtNDM4MS00NDRjLTkwNmQtOTkwMWMyMjhmMjdlXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_.jpg
All_Movie_Info: fantastic mr fox george clooney meryl streep bill murray jason schwartzman animation adventure comedy an urbane fox cannot resist returning to his farm raiding ways and then must help his community survive the farmers' retaliation


## Google Gemini API Yapısını Kullanarak LLM Tetikleme İşlemleri

- Düşük Değerler (0.1-0.4): Daha kesin ve daha tutarlı cevaplar verilir. Model daha tahmin edilebilir hale gelir.
- Orta Değerler (0.5-0.7): Hem mantıklı hem de yaratıcı cevaplar verilir.
- Yüksek Değerler (0.7-1): Daha rastgele ve yaratıcı, ancak bazen tutarsız yanıtlar verebilir.

In [21]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",  # Gemini 2.5 model
    temperature=0.3,  # Gemini 3.0+ defaults to 1.0
    max_tokens=500,
)

In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain


In [23]:
system_prompt = (
    "You are a helpful movie recommendation assistant. Use the following movie database context to answer the user's question."
    "Only recommend movies from the provided context"
    "Mention IMDB ratings and year where relevant."
    "If you don't find suitable movies in the context, say so honestly."
    "\n\n"
    "{context}"
)



In [24]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)


## Soru-Cevap Zinciri Oluşturma (LLM + PROMPT)


In [ ]:
question_answering_chain = create_stuff_documents_chain(llm, prompt)

## RAG Zinciri Oluşturma (RAG + LLM)


In [26]:
rag_chain = create_retrieval_chain(retriever, question_answering_chain)

## Kullanıcı sorgusunu çalıştırma

In [32]:
response = rag_chain.invoke({"input": "Movies over 2 hours long"})

print(response["answer"])

Here are some movies from the database that are over 2 hours long:

*   **The Lord of the Rings: The Two Towers** (2002) - 179 min, IMDb Rating: 8.8
*   **The Godfather: Part II** (1974) - 202 min, IMDb Rating: 9.0
*   **The Longest Day** (1962) - 178 min, IMDb Rating: 7.7
*   **2001: A Space Odyssey** (1968) - 149 min, IMDb Rating: 8.3
*   **25th Hour** (2002) - 135 min, IMDb Rating: 7.6
*   **Guardians of the Galaxy Vol. 2** (2017) - 136 min, IMDb Rating: 7.6
*   **Big Fish** (2003) - 125 min, IMDb Rating: 8.0
*   **Harry Potter and the Deathly Hallows: Part 2** (2011) - 130 min, IMDb Rating: 8.2
*   **Indiana Jones and the Last Crusade** (1989) - 127 min, IMDb Rating: 8.3
